# Pointers

A pointer is an object that stores the address of another object. If `T` is some type, then `T *` is a pointer that
can store the address an object of type `T`. Consider the following example:

```c
int *ptr;
```

`ptr` is a pointer object that can store the address an object of type `int`. Most programmers would say that
`ptr` is a pointer to `int`.

There is no universally agreed on style for declaring pointer variables. Each of the following are legal
ways of declaring a pointer:

```c
int *ptr1;
int* ptr2;
int * ptr3;
```

> A pointer references a location in memory, and obtaining the value stored at that location is known as dereferencing the pointer. As an analogy, a page number in a book's index could be considered a pointer to the corresponding page; dereferencing such a pointer would be done by flipping to the page with the given page number and reading the text found on that page.

#### `NULL`

The name `NULL` defined in the header `<stddef.h>` means "no object". Any pointer variable can be assigned 
a value of `NULL`:

```c
#include <stddef.h>

int *ptr = NULL;
```



#### A pointer should refer to an object

A pointer stores the address of an object but how does the programmer obtain the address of an object?
The *address of* operator `&` creates a pointer that refers to an object:

```c
int a = 99;
int *ptr = &a;
```

In the example above, `&a` yields a pointer to the `int` stored in the variable `a`, and that pointer
is then assigned to the variable `ptr`.

`ptr` can store a pointer to any `int` object so it is perfectly legal to re-assign which object is pointed at:


```c
int a = 99;
int *ptr = &a;

int b = 55;
ptr = &b;
```


#### Obtaining the value that the pointer refers to

The *pointer dereference* operator `*` dereferences a pointer to access the object that the pointer refers to:

```c
int a = 99;
int *ptr = &a;
int b = *ptr + 1;
```



In [1]:
// ptr1.c

#include <stdio.h>

int main(void) {
    int a = 99;
    int *ptr = &a;        // ptr refers to a
    int b = *ptr + 1      // b = a + 1
        
    printf("a = %d\n", a);
    printf("b = %d\n", b);
    
    return 0;
}

a = 99
b = 100


A value can be assigned to the object obtained via pointer dereferencing:

In [2]:
// ptr2.c 

#include <stdio.h>

int main(void) {
    int a = 99;
    int *ptr = &a;        // ptr refers to a
    *ptr = -33;           // assigns -33 to a
        
    printf("a = %d\n", a);
    
    return 0;
}

a = -33


Undefined behavior results if a null pointer is dereferenced:

In [4]:
// ptr3.c 

#include <stdio.h>

int main(void) {
    int *ptr = NULL;
    *ptr = -33;           // oops, null pointer dereference
        
    printf("a = %d\n", *ptr);
    
    return 0;
}

[C kernel] Executable exited with code -11

# Using pointers to solve a common programming problem

A common programming problem is writing a function to change the value of one of its parameters. A simple
example is a function that swaps the values its two parameters. A first attempt at such a function might
look like so:

In [3]:
// badswap.c

#include <stdio.h>

void swap(double x, double y) {
    double tmp = x;
    x = y;
    y = tmp;
    printf("inside swap: x = %f, y = %f\n", x, y);
}

int main(void) {
    double a = 1.5;
    double b = 99.9;
    
    printf("before calling swap: a = %f, b = %f\n", a, b);
    swap(a, b);
    printf("after calling swap: a = %f, b = %f\n", a, b);
    
    return 0;
}

before calling swap: a = 1.500000, b = 99.900000
inside swap: x = 99.900000, y = 1.500000
after calling swap: a = 1.500000, b = 99.900000


Running the example above illustrates that the `swap` function manages to swap the values of the parameters
`x` and `y` but does not swap the values of the arguments `a` and `b`. The reason this is occurs is because
C uses pass-by-value to transfer the values of the arguments to the parameters of the function. In pass-by-value,
the value of the arguments are copied into the parameters and the function manipulates the parameters. The values
of the parameters are *not* copied back to the arguments at the end of the function.

For `swap` to swap the values of the arguments, `swap` would need to receive the actual objects corresponding
to the arguments. The solution in C is to pass a pointer to the objects that should be swapped:

In [4]:
// swap.c

#include <stdio.h>

void swap(double *x, double *y) {
    double tmp = *x;    // tmp gets the value of the object pointed to by x
    *x = *y;            // the object pointed to by x gets the value of the object pointed to by y
    *y = tmp;           // the object pointed to by y gets the value of tmp
    printf("inside swap: x = %f, y = %f\n", *x, *y);
}

int main(void) {
    double a = 1.5;
    double b = 99.9;
    
    printf("before calling swap: a = %f, b = %f\n", a, b);
    swap(&a, &b);      // calls swap with the address of a and the address of b
    printf("after calling swap: a = %f, b = %f\n", a, b);
}

before calling swap: a = 1.500000, b = 99.900000
inside swap: x = 99.900000, y = 1.500000
after calling swap: a = 99.900000, b = 1.500000


Another example of using a pointer as a function parameter is in a function that needs to return more than
one value. For example, it is often useful to return the maximum value and the index of the maximum value
when finding the maximum value in an array. One way to do so is to return one value using the return value
and the second value is set by dereferencing a pointer passed in by the caller:

In [2]:
// max.c

#include <stdio.h>

// Returns the maximum value in the array a having count elements.
// The index of the maximum element is stored in the object pointed at
// by index.
int max(int a[], size_t count, size_t *index) {
    int hi = a[0];
    size_t i_hi = 0;
    for (size_t i = 1; i < count; i++) {
        if (a[i] > hi) {
            hi = a[i];
            i_hi = i;
        }
    }
    *index = i_hi;
    return hi;
}

int main(void) {
    int arr[] = { 6, 3, 1, 8, 7, 9, 4, 2, 5 };
    
    // must declare a variable to store the index in
    size_t i;
    int big = max(arr, 9, &i);
    
    printf("max = %d, index = %lu\n", big, i);
    
    return 0;
}

max = 9, index = 5


A common mistake made by new C programmers when using a function such as `max` is to use a pointer that
does not point at an actual object. In the example below, the programmer has used a pointer object as
the argument to `max` instead of using the address of an object:

In [4]:
// bad_call_to_max.c

#include <stdio.h>

// Returns the maximum value in the array a having count elements.
// The index of the maximum element is stored in the object pointed at
// by index.
int max(int a[], size_t count, size_t *index) {
    int hi = a[0];
    size_t i_hi = 0;
    for (size_t i = 1; i < count; i++) {
        if (a[i] > hi) {
            hi = a[i];
            i_hi = i;
        }
    }
    *index = i_hi;
    return hi;
}

int main(void) {
    int arr[] = { 6, 3, 1, 8, 7, 9, 4, 2, 5 };
    
    // oops, i does not point at a size_t object
    size_t *i;
    int big = max(arr, 9, i);
    
    printf("max = %d, index = %lu\n", big, i);
    
    return 0;
}

/tmp/tmp5calr9vw.c: In function ‘main’:
/tmp/tmp5calr9vw.c:26:15: warning: ‘i’ is used uninitialized [-Wuninitialized]
   26 |     int big = max(arr, 9, i);
      |               ^~~~~~~~~~~~~~
[C kernel] Executable exited with code -11

## Pointer arithmetic

